In [ ]:
import simpy
import numpy as np
import matplotlib.pyplot as plt

class AirportSimulation:
    def __init__(self, env, num_security_personnel, num_ticket_checkers, num_gates, gate_capacity, baggage_capacity, num_passengers, arrival_rate):
        self.env = env
        self.num_security_personnel = num_security_personnel
        self.num_ticket_checkers = num_ticket_checkers
        self.num_gates = num_gates
        self.gate_capacity = gate_capacity
        self.num_passengers = num_passengers
        self.arrival_rate = arrival_rate
        self.security = simpy.Resource(env, capacity=num_security_personnel)
        self.ticket_check = simpy.Resource(env, capacity=num_ticket_checkers)
        self.gates = [simpy.Resource(env, capacity=gate_capacity) for _ in range(num_gates)]
        self.baggage_container = simpy.Container(env, init=0, capacity=baggage_capacity)
        self.security_wait_times = []
        self.ticket_check_wait_times = []
        self.baggage_check_wait_times = []
        self.boarding_wait_times = []

    def passenger_flow(self):
        for i in range(self.num_passengers):
            arrival_time = np.random.exponential(1 / self.arrival_rate)
            yield self.env.timeout(arrival_time)
            self.env.process(self.passenger(i))

    def passenger(self, passenger_id):
        arrival_time = self.env.now
        print(f"Zaman {self.env.now:.2f}: Yolcu {passenger_id} havaalanına geldi")

        # Security Check
        security_start_time = self.env.now
        with self.security.request() as req:
            yield req
            yield self.env.timeout(np.random.triangular(1, 3, 5))
            security_end_time = self.env.now
            self.security_wait_times.append(security_end_time - security_start_time)
            print(f"Zaman {self.env.now:.2f}: Yolcu {passenger_id} güvenlik kontrolünden geçti")

        # Ticket Check
        ticket_check_start_time = self.env.now
        with self.ticket_check.request() as req:
            yield req
            yield self.env.timeout(np.random.triangular(1, 2, 4))
            ticket_check_end_time = self.env.now
            self.ticket_check_wait_times.append(ticket_check_end_time - ticket_check_start_time)
            print(f"Zaman {self.env.now:.2f}: Yolcu {passenger_id} bilet kontrolünden geçti")

        # Baggage Check
        baggage_check_start_time = self.env.now
        yield self.baggage_container.put(1)
        yield self.env.timeout(np.random.triangular(1, 3, 5))
        baggage_check_end_time = self.env.now
        self.baggage_check_wait_times.append(baggage_check_end_time - baggage_check_start_time)
        print(f"Zaman {self.env.now:.2f}: Yolcu {passenger_id} bagajını teslim etti")
        
        # Boarding the Plane
        boarding_start_time = self.env.now
        gate_number = passenger_id % self.num_gates
        request = self.gates[gate_number].request()
        yield request
        yield self.env.timeout(np.random.triangular(5, 10, 15))
        boarding_end_time = self.env.now
        self.boarding_wait_times.append(boarding_end_time - boarding_start_time)
        self.gates[gate_number].release(request)
        print(f"Zaman {self.env.now:.2f}: Yolcu {passenger_id} uçağa bindi")

env = simpy.Environment()

num_security_personnel = int(input("Güvenlik personeli sayısını girin: (önerilen giriş 2)"))
num_ticket_checkers = int(input("Bilet kontrol personeli sayısını girin: (önerilen giriş 2)"))
num_gates = int(input("Kapı sayısını girin: (önerilen giriş 3)"))
gate_capacity = int(input("Kapı kapasitesini girin: (önerilen giriş 50)"))
baggage_capacity = int(input("Bagaj kapasitesini girin: (önerilen giriş 100)"))
num_passengers = int(input("Yolcu sayısını girin: (önerilen giriş 100)"))
arrival_rate = float(input("Geliş oranını girin (dakikada yolcu sayısı): (önerilen giriş 1)"))

airport_sim = AirportSimulation(env, num_security_personnel, num_ticket_checkers, num_gates, gate_capacity, baggage_capacity, num_passengers, arrival_rate)

env.process(airport_sim.passenger_flow())
env.run()


data = [
    airport_sim.security_wait_times,
    airport_sim.ticket_check_wait_times,
    airport_sim.baggage_check_wait_times,
    airport_sim.boarding_wait_times
]

plt.figure(figsize=(10, 7))
plt.boxplot(data, patch_artist=True)
plt.xticks([1, 2, 3, 4], ['Güvenlik Kontrolü', 'Bilet Kontrolü', 'Bagaj Teslimi', 'Uçağa Biniş'])
plt.title('Her Aşamada Bekleme Süreleri')
plt.ylabel('Bekleme Süresi (dakika)')
plt.grid(True)
plt.show()
